# Data Preparation

`Overview`
This notebook handles the initial data processing pipeline:
- Loading raw data from source files
- Performing exploratory data analysis (EDA)
- Cleaning and handling missing values
- Feature preprocessing and engineering
- Exporting processed datasets for modeling

`Inputs`
- Raw data files from `../data/raw/` 

`Outputs`
- Processed datasets in `../data/processed/`
- EDA visualizations in `../reports/figures/`

`Dependencies`
- pandas
- numpy
- matplotlib
- seaborn

*Note: This is notebook 1 of the analysis pipeline*

In [2]:
# Imports 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
from pprint import pprint
from pathlib import Path
import csv
import sys

# Import custom modules
sys.path.append(os.path.abspath("../backend"))
from etl.ingestion import inspect_bad_lines, auto_fix_row, robust_csv_reader
from etl.cleaning import standardize_columns, clean_numeric_column, clean_date_column


AttributeError: module 'pandas' has no attribute 'dataFrame'

In [ ]:
!which python


/Users/bertdepoorter/opt/anaconda3/envs/MDA/bin/python


Here we load the project specific datasets as CSV files. In the follow-up cell, we load the auxiliary dataset containing extra information on the CORDIS-HORIZON projects. This includes
- Scientific vocabulary 
- legal basis documents
- organization
- project
- topics
- webItem 
- webLink

In [ ]:
run_dir = os.getcwd()
print(run_dir)

/Users/bertdepoorter/Nextcloud/EU_Horizon_Dashboard/notebooks


In [3]:
# Import the dataset as pandas DataFrame
run_dir = os.getcwd()
parent_dir = os.path.dirname(run_dir)

raw_dir = f'{parent_dir}/data/raw'
interim_dir = f'{parent_dir}/data/interim'
processed_dir = f'{parent_dir}/data/processed'

# define file paths to project-specific files
data_report_path = f'{raw_dir}/reportSummaries.csv'
data_filereport_path = f'{raw_dir}/file_report.csv'
data_publications_path = f'{raw_dir}/projectPublications.csv'
data_deliverables_path = f'{raw_dir}/projectDeliverables.csv'



## Define functions for cleaning
The following functions are necessary to load the datasets correctly without manually changing them.
- `inspect_bad_lines`: inspect lines that cannot be read directly
- `auto_fix_row`: function that fixes row by merging excess columns together
- `robust_csv_reader`: robust function that loads CSV files while applying `auto_fix_row` function on the bad lines

Usage:
```
# check bad lines
project_df, problematic_lines = inspect_bad_lines(project_path)

# INspect how many bad lines there are 
print(f"DataFrame loaded with {len(project_df)} rows.")
print(f"Number of problematic lines: {len(problematic_lines)}")
```

In [4]:
# function to check for missing values
def missing_values(data):
    # look for missing values
    report_missing = data.isnull()

    # check which columns are missing data
    for key in data:
        missing = report_missing[report_missing[key] == True]
        print(f'For key {key}:\n     {len(missing.id)} elements are missing.')

## Inspect Reports

In [5]:
# get DataFrame keys
data_report = pd.read_csv(data_report_path, delimiter=';')
data_report.keys()

Index(['id', 'title', 'projectID', 'projectAcronym', 'attachment',
       'contentUpdateDate', 'rcn'],
      dtype='object')

In [6]:
data_report.head()

,id,title,projectID,projectAcronym,attachment,contentUpdateDate,rcn
0,101066069_PSHORIZON,Periodic Reporting for period 1 - ERASMUS (Ear...,101066069,ERASMUS,NaN,2025-03-17 10:38:00,1267558
1,101073231_PSHORIZON,Periodic Reporting for period 1 - OncoProTools...,101073231,OncoProTools,/docs/results/horizon/101073/101073231_PS/2024...,2025-03-18 12:31:34,1270628
2,101068156_PSHORIZON,Periodic Reporting for period 1 - BLISS (Beta-...,101068156,BLISS,/docs/results/horizon/101068/101068156_PS/pict...,2025-03-05 11:47:45,1260626
3,101072180_PSHORIZON,Periodic Reporting for period 1 - Green2Ice (W...,101072180,Green2Ice,/docs/results/horizon/101072/101072180_PS/2023...,2025-02-14 10:36:27,1252991
4,101063407_PSHORIZON,Periodic Reporting for period 1 - GHost (His E...,101063407,GHost,/docs/results/horizon/101063/101063407_PS/pa-1...,2025-02-26 17:32:14,1257475


### Missing values
1. we check each column for missing values
2. Define decision tree for handling missing values
3. Change values algorithmically
4. Store update dataframe in interim directory


In [7]:
# look for missing values
report_missing = data_report.isnull()

# check which columns are missing data
for key in data_report:
    missing = report_missing[report_missing[key] == True]
    print(f'For key {key}:\n     {len(missing.id)} elements are missing.')

For key id:
     0 elements are missing.
For key title:
     0 elements are missing.
For key projectID:
     0 elements are missing.
For key projectAcronym:
     0 elements are missing.
For key attachment:
     1861 elements are missing.
For key contentUpdateDate:
     0 elements are missing.
For key rcn:
     0 elements are missing.


In [8]:
# fix missing attachments
data_report['attachment'] = data_report['attachment'].fillna('about:blank') 

# change the missing values in dataframe
project_reports_interim = data_report
# save updated dataframe to data/interim
project_reports_interim.to_csv(f'{interim_dir}/reportSummaries_interim.csv', sep=';')

### Inspect other report file
This CSV file does not contain useful information

In [9]:
data_filereport = pd.read_csv(data_filereport_path, delimiter=';')
data_filereport

,"filename,status, issue_cause downloadURL, issue_cause accessURL"
0,HORIZON Report summaries (individual XML files...
1,"HORIZON Projects,delivered,,"
2,"HORIZON Projects Deliverables,delivered,,"
3,"HORIZON Projects (individual XML files),delive..."
4,HORIZON Projects Deliverables (individual XML ...
5,"HORIZON Report summaries,delivered,,"
6,"HORIZON Publications,delivered,,"
7,"HORIZON Projects Deliverables,delivered,,"
8,"HORIZON Publications,delivered,,"
9,"HORIZON Projects Deliverables,delivered,,"


## Inspect deliverables

In [10]:
# Inspect Dataframe
# account for problematic lines

deliverables_df, problematic_lines = inspect_bad_lines(data_deliverables_path)

print(f"DataFrame loaded with {len(deliverables_df)} rows.")
print(f"Number of problematic lines: {len(problematic_lines)}")
    

Found 11 problematic lines. Displaying the first 5:
Line 1416: ['101081964_11_DELIVHORIZON', 'Guideline for trial implementation', 'Documents, reports', 'This guideline will summarise and present agricultural practices performed at BioMonitor4CAP farms and research sites with links to local/regional nature conservation goals and targeted species. Based on these baseline information the guideline will present how, when and to what extend research groups of WPs 2 and 4 are integrated into the activities of WP3. Supports specifically achieving following outcomes (Part B, section 2.3): A and B ""Strategy on monitoring soil biodiversity at farm scale adopted by science, end users, and policy"" and E ""Roadmap on expanding enhancing application and implementation of agri-environmental measures committing to the preservation of biodiversity', ' particularly agroforestry."" This deliverable is an output of task 3.1.""', '101081964', 'BioMonitor4CAP', 'https://ec.europa.eu/research/participants

In [11]:
# Try loaoding with the robust CSV rreader
data_deliverables = robust_csv_reader(data_deliverables_path, expected_columns=10, problematic_column=3)


In [13]:
bad_line_no = []
for i in range(len(problematic_lines)):
    bad_line_no.append(problematic_lines[i][0])
bad_line_no

[1416, 1424, 6677, 6678, 8812, 8826, 9360, 9524, 10128, 13108, 19931]

In [14]:
data_deliverables.loc[bad_line_no]

,id,title,deliverableType,description,projectID,projectAcronym,url,collection,contentUpdateDate,rcn
1416,101095754_17_DELIVHORIZON,Preliminary report on population exposure maps,"Documents, reports",Preliminary report on population exposure maps...,101095754,NEEDED,https://ec.europa.eu/research/participants/doc...,Project deliverable,2024-12-09 12:38:47,1227504
1424,101095634_19_DELIVHORIZON,ENTRUST Website design and release,"Websites, patent fillings, videos etc.",The release of the ENTRUST Website as well as ...,101095634,ENTRUST,https://ec.europa.eu/research/participants/doc...,Project deliverable,2024-10-29 20:07:42,1183851
6677,101061015_5_DELIVHORIZON,Report on upscaling of WT microalgae strain,"Demonstrators, pilots, prototypes","Deliverable 3.1 """"Report on the upscaling of W...",101061015,GeneBEcon,https://ec.europa.eu/research/participants/doc...,Project deliverable,2024-08-06 15:04:04,1142319
6678,101061015_6_DELIVHORIZON,Preliminary toolbox for gene editing,"Documents, reports","Deliverable 2.1 """"Preliminary toolbox for gene...",101061015,GeneBEcon,https://ec.europa.eu/research/participants/doc...,Project deliverable,2024-08-06 15:04:26,1142272
8812,101116996_1_DELIVHORIZON,Research Data management,Data Management Plan,Drafting of Data Management plan,101116996,TEMPRODROME,https://ec.europa.eu/research/participants/doc...,Project deliverable,2024-07-22 13:49:46,1131743
8826,101079392_20_DELIVHORIZON,One Video News Releases,"Websites, patent fillings, videos etc.",One Video News Releases,101079392,HybridNeuro,https://ec.europa.eu/research/participants/doc...,Project deliverable,2024-07-16 16:32:47,1128939
9360,101075460_19_DELIVHORIZON,D5.4 Updated web resources/signposting,"Websites, patent fillings, videos etc.",Ensure updated information and signposting on ...,101075460,InspireuropePlus,https://ec.europa.eu/research/participants/doc...,Project deliverable,2024-07-01 13:20:44,1124352
9524,101126554_7_DELIVHORIZON,Call 1 – Link to Euraxess website,Other,Inform the Project Officer on publishing each ...,101126554,MathPhDInFrance,https://ec.europa.eu/research/participants/doc...,Project deliverable,2024-06-17 13:33:21,1111112
10128,101087249_1_DELIVHORIZON,Data Management Plan,Data Management Plan,An initial version of the Research Data Manage...,101087249,DISPATCHES,https://ec.europa.eu/research/participants/doc...,Project deliverable,2024-10-29 13:18:10,1177265
13108,101069525_15_DELIVHORIZON,Demonstration implementation plans,"Documents, reports",Demonstration implementation plans.The task wi...,101069525,TRANS-SAFE,https://ec.europa.eu/research/participants/doc...,Project deliverable,2024-04-08 17:16:38,1076003


### Missing values
Here we handle the missing values in the dataset

In [15]:
# look for missing values
report_missing = data_deliverables.isnull()

# check which columns are missing data
for key in data_deliverables:
    missing = report_missing[report_missing[key] == True]
    print(f'For key {key}:\n     {len(missing.id)} elements are missing.')

For key id:
     0 elements are missing.
For key title:
     0 elements are missing.
For key deliverableType:
     0 elements are missing.
For key description:
     0 elements are missing.
For key projectID:
     0 elements are missing.
For key projectAcronym:
     0 elements are missing.
For key url:
     0 elements are missing.
For key collection:
     0 elements are missing.
For key contentUpdateDate:
     0 elements are missing.
For key rcn:
     0 elements are missing.


In [16]:
# look for missing values
deliverables_missing = data_deliverables.isnull()

# check which columns are missing data
for key in data_deliverables:
    missing = deliverables_missing[deliverables_missing[key] == True]
    print(f'For key {key}:\n     {len(missing.id)} elements are missing.')

For key id:
     0 elements are missing.
For key title:
     0 elements are missing.
For key deliverableType:
     0 elements are missing.
For key description:
     0 elements are missing.
For key projectID:
     0 elements are missing.
For key projectAcronym:
     0 elements are missing.
For key url:
     0 elements are missing.
For key collection:
     0 elements are missing.
For key contentUpdateDate:
     0 elements are missing.
For key rcn:
     0 elements are missing.


We are missing elements in the following columns:
- deliverableType
    - option 1: change to `'other'`
    - option 2: look up individual titles and add manually
- description
    - option 1: add empty string
    - Inspect manually to gain more insight what they exactly represent
        - Update: all the titles related to the projects are quite related. I suggest we copy title values into the description column.
- url
    - 1 missing url. Add the url to the main page of this project (SELFY, id = 101069748_16_DELIVHORIZON) instead of link to deliverable?
- rcn
    - 1 rcn is missing. 
    - Looked this number up in publication list based on the projectAcronym = `'GeneBEcon'`. There the rcn number is gives as `1077637.0`


In [17]:
# change unknown deliverable types to other
data_deliverables['deliverableType'] = data_deliverables['deliverableType'].fillna('Other') 

# change empty descriptions to title of that particular row
data_deliverables['description'] = data_deliverables['description'].fillna(data_deliverables['title'])

# change missing url to homepage of the particular project
data_deliverables['url'] = data_deliverables['url'].fillna('https://selfy-project.eu/')

# add missing rcn number
data_deliverables['rcn'] = data_deliverables['rcn'].fillna(1077637.0)

In [18]:
# check whether filling executed correctly
data_deliverables[deliverables_missing.deliverableType == True]

,id,title,deliverableType,description,projectID,projectAcronym,url,collection,contentUpdateDate,rcn


In [19]:
# save updated dataframe to data/interim
data_deliverables.to_csv(f'{interim_dir}/projectdeliverables_interim.csv', sep=';')

In [20]:
data_deliverables.keys()

Index(['id', 'title', 'deliverableType', 'description', 'projectID',
       'projectAcronym', 'url', 'collection', 'contentUpdateDate', 'rcn'],
      dtype='object')

## Inspect Publications

In [21]:
# Inspect Dataframe
data_publications = pd.read_csv(data_publications_path, delimiter=';')
data_publications.keys()

Index(['id', 'title', 'isPublishedAs', 'authors', 'journalTitle',
       'journalNumber', 'publishedYear', 'publishedPages', 'issn', 'isbn',
       'doi', 'projectID', 'projectAcronym', 'collection', 'contentUpdateDate',
       'rcn'],
      dtype='object')

In [22]:
publications_df, problematic_lines = inspect_bad_lines(data_publications_path, expected_columns=16)

print(f"DataFrame loaded with {len(publications_df)} rows.")
print(f"Number of problematic lines: {len(problematic_lines)}")

DataFrame loaded with 24150 rows.
Number of problematic lines: 0


Some entries in the publications CSV have been changed by hand, in order to allow loading them:
- 7588
- 7748

Both are from the same conference. Problem: switched the order of the columns and add one additional empty column causing pandas loader to crash. 

Next problems:
- 12036: wrong notation of authors names + use of ; delimiter inside string.
- 12043: start authors string with four " + use ; to separate names.
- 12099: same problem as stated above
- 12110: same problem
- 12115: same problem
- 12270: same problem
- 18735: same problem
- 24019: same problem




In [23]:
data_publications

,id,title,isPublishedAs,authors,journalTitle,journalNumber,publishedYear,publishedPages,issn,isbn,doi,projectID,projectAcronym,collection,contentUpdateDate,rcn
0,101040480_113381_PUBLIHORIZON,The Microwave Rotational Electric Resonance (R...,Peer reviewed articles,"Hamza El Hadki, Kenneth J. Koziol, Oum Keltoum...",Molecules,28,2023,NaN,1420-3049,NaN,10.3390/molecules28083419,101040480,LACRIDO,Project publication,2025-02-11 11:41:50,1243351
1,101040480_113371_PUBLIHORIZON,The microwave spectra of the conformers of n-b...,Peer reviewed articles,"Susanna L. Stephens, Eléonore Antonelli, Alexa...",Journal of Molecular Spectroscopy,397,2024,NaN,0022-2852,NaN,10.1016/j.jms.2023.111824,101040480,LACRIDO,Project publication,2025-02-11 11:01:16,1243327
2,101040480_113383_PUBLIHORIZON,Coupled internal rotations and 14N quadrupole ...,Peer reviewed articles,"Mike Barth, Isabelle Kleiner, Ha Vinh Lam Nguyen",The Journal of Chemical Physics,160,2024,NaN,0021-9606,NaN,10.1063/5.0213319,101040480,LACRIDO,Project publication,2025-02-11 11:40:00,1243350
3,101040480_113375_PUBLIHORIZON,"The Heavy Atom Structure, <i>“cis</i> effect” ...",Peer reviewed articles,"Truong Anh Nguyen, Isabelle Kleiner, Martin Sc...",ChemPhysChem,25,2024,NaN,1439-4235,NaN,10.1002/cphc.202400387,101040480,LACRIDO,Project publication,2025-02-11 11:11:17,1243343
4,101040480_113374_PUBLIHORIZON,"Structure determination of 2,5-difluorophenol ...",Peer reviewed articles,"K.P. Rajappan Nair, Kevin G. Lengsfeld, Philip...",Journal of Molecular Structure,1321,2024,NaN,0022-2860,NaN,10.1016/j.molstruc.2024.139971,101040480,LACRIDO,Project publication,2025-02-11 11:04:26,1243340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24145,101060809_11542_PUBLIHORIZON,Review of International Studies,Peer reviewed articles,"Pogdda, S., Richmond, O., Visoka, G.",Review of International Studies,3,2023,NaN,0260-2105,NaN,10.1017/S0260210522000377,101060809,EMBRACE,Project publication,2024-02-13 10:14:03,1042445
24146,101061890_16827_PUBLIHORIZON,Dual-band electro-optically steerable antenna,Peer reviewed articles,"Dmytro Vovchuk, Anna Mikhailovskaya, Dmitry Do...",Journal of Optics,25 105601,2023,NaN,2040-8986,NaN,10.1088/2040-8986/acf1ae,101061890,DeepSight,Project publication,2024-02-19 09:22:09,1043146
24147,101061201_13995_PUBLIHORIZON,Positron Annihilation Study of RPV Steels Radi...,Peer reviewed articles,Vladimir Slugen; Tomas Brodziansky; Jana Simeg...,Materials; Volume 15; Issue 20; Pages: 7091,NaN,2022,NaN,1996-1944,NaN,10.3390/ma15207091,101061201,DELISA- LTO,Project publication,2024-01-22 18:22:57,1035198
24148,101061201_13826_PUBLIHORIZON,Round Robin Tests for WWER Heat Exchange Tubes,Peer reviewed articles,"Roman Krajcovic, Michal Benak, Radim Kopriva, ...",e-Journal of Nondestructive Testing 28(7),NaN,2023,NaN,1435-4934,NaN,10.58286/28273,101061201,DELISA- LTO,Project publication,2024-01-16 10:28:00,1033841


### Missin values
Here we inspect the missing data in this file, and outline how we are goiing to treat these missing data points

In [24]:
# look for missing values
publications_missing = data_publications.isnull()

# check which columns are missing data
for key in publications_missing.keys():
    missing = publications_missing[publications_missing[key] == True]
    if len(missing.id) > 0:
        print(f'For key {key}:\n     {len(missing.id)} elements are missing.')

For key authors:
     75 elements are missing.
For key journalTitle:
     2099 elements are missing.
For key journalNumber:
     13622 elements are missing.
For key publishedPages:
     24142 elements are missing.
For key issn:
     7004 elements are missing.
For key isbn:
     23219 elements are missing.
For key doi:
     2293 elements are missing.


There is quite some missing data in this file. Let us go through each line individually.
- authors:
    - This sucks. Would have been very nice to decompose author strings into single authors and make the connections
    - How to treat this: look into the article title string to check whether this one contains more author infromation
- journalTitle:
    - chack in the publication title. Sometimes there one has just copy-pasted the whole article reference
- journalNumber:
    - Not the most relevant parameter in my opinion. Just make all NaN zeros
- publishedYear:
    - Manually look this up
- publishedPages:
    - Not the most relevant parameter in my opinion. Just make all NaN zeros
- issn:
    - Not the most relevant parameter in my opinion. Just make all NaN zeros
- isbn:
    - Not the most relevant parameter in my opinion. Just make all NaN zeros
- doi:
    - Fuck this, just pass about:blank as url. 
- rcn:
    - Manually adjust this one. 
        - Update: this entry was missing an entry for authors, all following field shifted 1 column to the left. Manually fixed this one. 



In [25]:
data_publications.keys()

Index(['id', 'title', 'isPublishedAs', 'authors', 'journalTitle',
       'journalNumber', 'publishedYear', 'publishedPages', 'issn', 'isbn',
       'doi', 'projectID', 'projectAcronym', 'collection', 'contentUpdateDate',
       'rcn'],
      dtype='object')

In [26]:
# check missing rcn. 
data_publications[publications_missing.rcn == True]

,id,title,isPublishedAs,authors,journalTitle,journalNumber,publishedYear,publishedPages,issn,isbn,doi,projectID,projectAcronym,collection,contentUpdateDate,rcn


The cell above does give an empty DataFrame. Used it to get the information to look up rcn number in other datasets.

In [27]:
# fill some gaps in the data structure
data_publications['isbn'] = data_publications['isbn'].fillna('0000-0000')
data_publications['issn'] = data_publications['issn'].fillna('0000-0000')
data_publications['publishedPages'] = data_publications['publishedPages'].fillna(0)
data_publications['doi'] = data_publications['doi'].fillna('about:blank')
data_publications['journalTitle'] = data_publications['journalTitle'].fillna('Miscalleneous')
data_publications['journalNumber'] = data_publications['journalNumber'].fillna(0)
data_publications['authors'] = data_publications['authors'].fillna('sine nome')


In [28]:
# check data_publications again
publications_missing = data_publications.isnull()

# check which columns are missing data
for key in publications_missing.keys():
    missing = publications_missing[publications_missing[key] == True]
    if len(missing.id) > 0:
        print(f'For key {key}:\n     {len(missing.id)} elements are missing.')

Now there are no empty entries left. We store the completed dataset in the interim folder

In [39]:
# Save to intermediate
data_publications.to_csv(f'{interim_dir}/projectPublications_interim.csv', sep=';')

## Inspect CORDIS-HORIZON projects data files
This is the folder containing some more datasets on the different projects.

In [29]:
# define file paths
SciVoc_path = f'{raw_dir}/euroSciVoc.csv'
legalBasis_path = f'{raw_dir}/legalBasis.csv'
organization_path = f'{raw_dir}/organization.csv'
project_path = f'{raw_dir}/project.csv'
topics_path = f'{raw_dir}/topics.csv'
webItems_path = f'{raw_dir}/webItem.csv'
webLink_path = f'{raw_dir}/webLink.csv'

In [30]:
# print keys for each file
def print_keys(file_path):
    df = pd.read_csv(file_path, delimiter=';')
    print(f"Keys for {file_path}:")
    print(df.keys())
    print("\n")

In [31]:
# Import some informative files

# load datasets
read_csv_options = {
    "delimiter": ";",
    "quotechar": '"',
    "escapechar": "\\",
    'doublequote': False,
    # "on_bad_lines": "skip",   # we skip lines that do not import properly for now
    "engine": "python"  # 'python' engine handles complex parsing better
}


sci_voc_df = pd.read_csv(SciVoc_path, **read_csv_options)
legal_basis_df = pd.read_csv(legalBasis_path, **read_csv_options)
organization_df = pd.read_csv(organization_path, delimiter=';')
topics_df = pd.read_csv(topics_path, **read_csv_options)
web_items_df = pd.read_csv(webItems_path, **read_csv_options)
web_link_df = pd.read_csv(webLink_path, **read_csv_options)


In [32]:
sci_voc_df.keys()

Index(['projectID', 'euroSciVocCode', 'euroSciVocPath', 'euroSciVocTitle',
       'euroSciVocDescription'],
      dtype='object')

In [33]:
sci_voc_df.head()

,projectID,euroSciVocCode,euroSciVocPath,euroSciVocTitle,euroSciVocDescription
0,101159220,/21/33/121/621,/medical and health sciences/health sciences/i...,malaria,NaN
1,101159220,/23/49/315/997/1613,/natural sciences/biological sciences/biochemi...,proteins,NaN
2,101093997,/23/49/315/997/1611,/natural sciences/biological sciences/biochemi...,carbohydrates,NaN
3,101126531,/23/49/341/325,/natural sciences/biological sciences/microbio...,virology,NaN
4,101126531,/21/33/121/44109686/8132740,/medical and health sciences/health sciences/i...,coronaviruses,NaN


In [34]:
sci_voc_df.head()
# check data_publications again
sci_voc_missing = sci_voc_df.isnull()

# check which columns are missing data
for key in sci_voc_df.keys():
    missing = sci_voc_missing[sci_voc_missing[key] == True]
    if len(missing.projectID) > 0:
        print(f'For key {key}:\n     {len(missing.projectID)} elements are missing.')

For key euroSciVocDescription:
     34612 elements are missing.


In [35]:
len(sci_voc_df['euroSciVocDescription']) #.unique()

34612

In [36]:
legal_basis_df


,projectID,legalBasis,title,uniqueProgrammePart
0,101159220,HORIZON.2.1,Health,True
1,101096150,HORIZON.3.2,European innovation ecosystems,True
2,101096150,HORIZON.3.2.3,Joint programmes close to innovators,NaN
3,101093997,HORIZON.2.1,Health,True
4,101093997,HORIZON.2.1.1,Health throughout the Life Course,NaN
...,...,...,...,...
21031,101052247,HORIZON.4.2,Reforming and enhancing the European R&I System,True
21032,101052247,HORIZON.4.2.4,Open science,NaN
21033,101172981,HORIZON.4.2.6,Careers and universities,NaN
21034,101172981,HORIZON.4.2,Reforming and enhancing the European R&I System,True


In [37]:
legal_missing = legal_basis_df.isnull()

# check which columns are missing data
for key in legal_basis_df.keys():
    missing = legal_missing[legal_missing[key] == True]
    if len(missing.projectID) > 0:
        print(f'For key {key}:\n     {len(missing.projectID)} elements are missing.')

For key uniqueProgrammePart:
     5173 elements are missing.


In [38]:
legal_basis_df['uniqueProgrammePart'].unique()

array([True, nan], dtype=object)

In [39]:
organization_df

,projectID,projectAcronym,organisationID,vatNumber,name,shortName,SME,activityType,street,postCode,...,contactForm,contentUpdateDate,rcn,order,role,ecContribution,netEcContribution,totalCost,endOfParticipation,active
0,101159220,PvSeroRDT,986872084,NaN,Institut Pasteur de Madagascar,IPM,False,REC,Ambatofotsikely - Avaradoha,101,...,https://ec.europa.eu/info/funding-tenders/oppo...,2024-12-24 11:18:48,1947090,4,associatedPartner,NaN,0.00,0,False,NaN
1,101159220,PvSeroRDT,999542806,NaN,INSTITUT PASTEUR DE DAKAR,NaN,False,REC,AVENUE PASTEUR 36,DAKAR,...,https://ec.europa.eu/info/funding-tenders/oppo...,2024-12-24 11:18:48,1906512,2,participant,1777625.00,1777625.00,1777625,False,NaN
2,101159220,PvSeroRDT,889740358,NaN,ARMAUER HANSEN RESEARCH INSTITUTE,NaN,False,REC,JIMMA ROAD ALERT COMPOUND,1005,...,https://ec.europa.eu/info/funding-tenders/oppo...,2024-12-24 11:18:48,1975141,3,participant,272174.38,272174.38,"272174,38",False,NaN
3,101159220,PvSeroRDT,999912667,GB233756066,LONDON SCHOOL OF HYGIENE AND TROPICAL MEDICINE...,LSHTM,False,HES,KEPPEL STREET,WC1E 7HT,...,https://ec.europa.eu/info/funding-tenders/oppo...,2024-12-24 11:18:48,1906028,6,associatedPartner,NaN,0.00,0,False,NaN
4,101159220,PvSeroRDT,999679964,NaN,FOUNDATION FOR INNOVATIVE NEW DIAGNOSTICS,"FIND, the global alliance for diagnostics",False,OTH,"CAMPUS BIOTECH, CHEMIN DES MINES 9",1202,...,https://ec.europa.eu/info/funding-tenders/oppo...,2024-12-24 11:18:48,1906308,7,associatedPartner,NaN,0.00,0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101148,101172981,EUCYS2024,999917614,PL6340197134,UNIWERSYTET SLASKI W KATOWICACH,UNIWERSYTET SLASKI W KATOWICACH,False,HES,UL. BANKOWA 12,40-007,...,https://ec.europa.eu/info/funding-tenders/oppo...,2024-04-22 17:56:03,1906340,1,coordinator,999500.00,999500.00,999500,False,NaN
101149,101131799,CO-VALUE,899261102,IT03786281208,ART-ER-SOCIETA CONSORTILE PER AZIONI,ART-ER,False,OTH,VIA PIERO GOBETTI 101,40129,...,https://ec.europa.eu/info/funding-tenders/oppo...,2023-11-08 18:09:10,1958836,4,participant,196250.00,196250.00,196250,False,NaN
101150,101131799,CO-VALUE,896926700,BE0745750648,STICKYDOT SRL,STICKYDOT SRL,True,PRC,RUE DU MONASTERE 10,1000,...,https://ec.europa.eu/info/funding-tenders/oppo...,2023-11-08 18:09:10,1967836,1,coordinator,355750.00,355750.00,355750,False,NaN
101151,101131799,CO-VALUE,940963536,IT12520270153,FONDAZIONE GIANNINO BASSETTI ETS,FGB,False,REC,VIA MICHELE BAROZZI 4,20122,...,https://ec.europa.eu/info/funding-tenders/oppo...,2023-11-08 18:09:10,2517504,2,participant,259000.00,259000.00,259000,False,NaN


In [40]:
organization_df['active'].unique()

array([nan])

In [41]:
organization_missing = organization_df.isnull()

# check which columns are missing data
for key in organization_df.keys():
    missing = organization_missing[organization_missing[key] == True]
    if len(missing.projectID) > 0:
        print(f'For key {key}:\n     {len(missing.projectID)} elements are missing.')

For key vatNumber:
     15490 elements are missing.
For key shortName:
     25689 elements are missing.
For key SME:
     263 elements are missing.
For key activityType:
     23 elements are missing.
For key street:
     305 elements are missing.
For key postCode:
     786 elements are missing.
For key city:
     263 elements are missing.
For key country:
     13 elements are missing.
For key nutsCode:
     278 elements are missing.
For key geolocation:
     673 elements are missing.
For key organizationURL:
     39135 elements are missing.
For key ecContribution:
     15840 elements are missing.
For key netEcContribution:
     23 elements are missing.
For key totalCost:
     599 elements are missing.
For key active:
     101153 elements are missing.


In [53]:
organization_df[organization_missing.city == True]
# organization_df['country'][organization_missing]

,projectID,projectAcronym,organisationID,vatNumber,name,shortName,SME,activityType,street,postCode,...,contactForm,contentUpdateDate,rcn,order,role,ecContribution,netEcContribution,totalCost,endOfParticipation,active
134,101104469,SALVOVAR,885311047,NaN,SUSANNAH CARROLL,SUSANNAH CARROLL,NaN,PRC,NaN,NaN,...,https://ec.europa.eu/info/funding-tenders/oppo...,2023-06-29 16:17:12,1981816,9,participant,220875.00,220875.00,220875,False,NaN
451,101045949,XVR011 Phase 2,920482568,NaN,GRANZER HEINZ ULRICH,GRANZER REGULATORY CONSULTING & SERVICES,NaN,PRC,NaN,NaN,...,https://ec.europa.eu/info/funding-tenders/oppo...,2022-07-27 18:13:40,3146150,3,participant,387500.00,387500.00,387500,False,NaN
711,101186869,AGRI-BIOCIRCULAR-HUB,887616058,NaN,Marcin Chmielarz,Hurtownia Tworzyw Sztucznych GRANULAT BIS Marc...,NaN,PRC,NaN,NaN,...,https://ec.europa.eu/info/funding-tenders/oppo...,2024-11-08 16:30:32,1990445,13,participant,145420.00,145420.00,145420,False,NaN
1122,101080267,NEXGEN-PD,920482568,NaN,GRANZER HEINZ ULRICH,GRANZER REGULATORY CONSULTING & SERVICES,NaN,PRC,NaN,NaN,...,https://ec.europa.eu/info/funding-tenders/oppo...,2023-08-29 02:11:16,3146150,8,participant,701250.00,701250.00,701250,False,NaN
1254,101121171,SHIELD4CROWD,884584420,NaN,BOUALI YOUSSEF,DIGINNOV - DIGITAL INNOVATION CONSULTING DI YO...,NaN,PRC,NaN,NaN,...,https://ec.europa.eu/info/funding-tenders/oppo...,2023-08-10 18:06:47,1983510,2,participant,59229.20,24739.60,0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98561,101136006,XTREME,882775564,NaN,ANE KRUSE SKAK,IMMERSIVE STORIES,NaN,PRC,NaN,NaN,...,https://ec.europa.eu/info/funding-tenders/oppo...,2023-11-21 16:53:20,1985274,9,participant,224265.00,224265.00,224265,False,NaN
98811,101072892,LongTREC,934023477,NaN,Stichting Global Organisation for Bioinformati...,GOBLET,NaN,OTH,NaN,NaN,...,https://ec.europa.eu/info/funding-tenders/oppo...,2022-09-12 16:32:58,1977126,23,associatedPartner,NaN,0.00,0,False,NaN
98854,101073088,MAWI,916290810,NaN,HERTZ BROERTJES BRIGITTE,NaN,NaN,PRC,NaN,NaN,...,https://ec.europa.eu/info/funding-tenders/oppo...,2022-09-15 14:53:19,1949833,17,associatedPartner,NaN,0.00,0,False,NaN
100617,101046182,iMPact,888342103,NaN,BLUST VOLKER,PHOX Consulting e.U.,NaN,PRC,NaN,NaN,...,https://ec.europa.eu/info/funding-tenders/oppo...,2022-09-26 10:28:07,1978078,5,participant,30945.39,30945.39,"30945,39",False,NaN


In [46]:
from etl.cleaning import get_country_from_city

In [51]:
inferred_countries = get_country_from_city('Windhoek')

In [52]:
type(inferred_countries)

NoneType

In [ ]:
# Treat missing values
organization_df['vatNumber'] = organization_df['vatNumber'].fillna('XX000000000')
organization_df['shortName'] = organization_df['shortName'].fillna('XX')
organization_df['SME'] = organization_df['SME'].fillna('XX')
organization_df['activityType'] = organization_df['activityType'].fillna('XX')
organization_df['street'] = organization_df['street'].fillna('XX')
organization_df['city'] = organization_df['city'].fillna('XX')

organization_df['country'] = organization_df['country'].fillna('NB')
organization_df['nutsCode'] = organization_df['nutsCode'].fillna('XX')
# Try to find geolocation from city

organization_df['geolocation'] = organization_df['geolocation'].fillna('XX')

organization_df['organizationURL'] = organization_df['organizationURL'].fillna('about:blank')
organization_df['ecContribution'] = organization_df['ecContribution'].fillna(0.0)
organization_df['netEcContribution'] = organization_df['netEcContribution'].fillna(0.0)

organization_df['totalCost'] = organization_df['totalCost'].fillna(0.0)





NameError: name 'project_df' is not defined

In [83]:
# Merge project_df (with 'id') into organization_df (with 'projectID')
org_with_status = organization_df.merge(
    project_df[['id', 'status']],
    left_on='projectID',
    right_on='id',
    how='left'
)

# from etl.cleaning import 
# Set 'active' to True if status is SIGNED, else False
org_with_status['active'] = org_with_status['status'].astype(str).str.encode('unicode_escape').str.decode('utf-8').str.strip() == 'SIGNED'

org_with_status = org_with_status.drop(columns=['id', 'status'])

In [82]:
org_with_status['status'].astype(str).str.encode('unicode_escape').str.decode('utf-8').str.strip() == 'SIGNED'

0         True
1         True
2         True
3         True
4         True
          ... 
101148    True
101149    True
101150    True
101151    True
101152    True
Name: status, Length: 101153, dtype: bool

In [84]:
org_with_status

,projectID,projectAcronym,organisationID,vatNumber,name,shortName,SME,activityType,street,postCode,...,contactForm,contentUpdateDate,rcn,order,role,ecContribution,netEcContribution,totalCost,endOfParticipation,active
0,101159220,PvSeroRDT,986872084,NaN,Institut Pasteur de Madagascar,IPM,False,REC,Ambatofotsikely - Avaradoha,101,...,https://ec.europa.eu/info/funding-tenders/oppo...,2024-12-24 11:18:48,1947090,4,associatedPartner,NaN,0.00,0,False,True
1,101159220,PvSeroRDT,999542806,NaN,INSTITUT PASTEUR DE DAKAR,NaN,False,REC,AVENUE PASTEUR 36,DAKAR,...,https://ec.europa.eu/info/funding-tenders/oppo...,2024-12-24 11:18:48,1906512,2,participant,1777625.00,1777625.00,1777625,False,True
2,101159220,PvSeroRDT,889740358,NaN,ARMAUER HANSEN RESEARCH INSTITUTE,NaN,False,REC,JIMMA ROAD ALERT COMPOUND,1005,...,https://ec.europa.eu/info/funding-tenders/oppo...,2024-12-24 11:18:48,1975141,3,participant,272174.38,272174.38,"272174,38",False,True
3,101159220,PvSeroRDT,999912667,GB233756066,LONDON SCHOOL OF HYGIENE AND TROPICAL MEDICINE...,LSHTM,False,HES,KEPPEL STREET,WC1E 7HT,...,https://ec.europa.eu/info/funding-tenders/oppo...,2024-12-24 11:18:48,1906028,6,associatedPartner,NaN,0.00,0,False,True
4,101159220,PvSeroRDT,999679964,NaN,FOUNDATION FOR INNOVATIVE NEW DIAGNOSTICS,"FIND, the global alliance for diagnostics",False,OTH,"CAMPUS BIOTECH, CHEMIN DES MINES 9",1202,...,https://ec.europa.eu/info/funding-tenders/oppo...,2024-12-24 11:18:48,1906308,7,associatedPartner,NaN,0.00,0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101148,101172981,EUCYS2024,999917614,PL6340197134,UNIWERSYTET SLASKI W KATOWICACH,UNIWERSYTET SLASKI W KATOWICACH,False,HES,UL. BANKOWA 12,40-007,...,https://ec.europa.eu/info/funding-tenders/oppo...,2024-04-22 17:56:03,1906340,1,coordinator,999500.00,999500.00,999500,False,True
101149,101131799,CO-VALUE,899261102,IT03786281208,ART-ER-SOCIETA CONSORTILE PER AZIONI,ART-ER,False,OTH,VIA PIERO GOBETTI 101,40129,...,https://ec.europa.eu/info/funding-tenders/oppo...,2023-11-08 18:09:10,1958836,4,participant,196250.00,196250.00,196250,False,True
101150,101131799,CO-VALUE,896926700,BE0745750648,STICKYDOT SRL,STICKYDOT SRL,True,PRC,RUE DU MONASTERE 10,1000,...,https://ec.europa.eu/info/funding-tenders/oppo...,2023-11-08 18:09:10,1967836,1,coordinator,355750.00,355750.00,355750,False,True
101151,101131799,CO-VALUE,940963536,IT12520270153,FONDAZIONE GIANNINO BASSETTI ETS,FGB,False,REC,VIA MICHELE BAROZZI 4,20122,...,https://ec.europa.eu/info/funding-tenders/oppo...,2023-11-08 18:09:10,2517504,2,participant,259000.00,259000.00,259000,False,True


In [81]:
organization_df['active'].unique()

array([nan])

In [55]:
topics_df.head()

,projectID,topic,title
0,101159220,HORIZON-JU-GH-EDCTP3-2023-02-02-two-stage,Advancing point-of-care diagnostics to the market
1,101096150,HORIZON-EIE-2022-CONNECT-01-01,Towards more inclusive networks and initiative...
2,101093997,HORIZON-HLTH-2022-STAYHLTH-02-01,Personalised blueprint of chronic inflammation...
3,101126531,HORIZON-HLTH-2022-CEPI-15-01-IBA,CEPI 4 - Contribution to the Coalition for Epi...
4,101113979,HORIZON-EIE-2022-SCALEUP-02-02,Women TechEU


In [56]:
topics_missing = topics_df.isnull()

# check which columns are missing data
for key in topics_df.keys():
    missing = topics_missing[topics_missing[key] == True]
    if len(missing.projectID) > 0:
        print(f'For key {key}:\n     {len(missing.projectID)} elements are missing.')

In [60]:
web_items_df

,language,availableLanguages,uri,title,type,source,represents
0,en,en,/docs/projects/files/101/101119806/251305.jpg,NaN,relatedImage,editorial,projectLogo
1,en,"en,any",/docs/project/press-release-kom-before-event-d...,PRESS RELEASE CARE4CARE Kick-Off Meeting,relatedPrint,editorial,NaN
2,any,any,/docs/projects/files/101/101095303/243751.png,NaN,relatedImage,editorial,projectLogo
3,en,"en,any",/docs/projects/files/101/101075659/240868.jpg,NaN,relatedImage,editorial,projectLogo


In [59]:
web_link_df

,projectID,physUrl,id,availableLanguages,status,archivedDate,type,source,represents
0,101096150,https://ec.europa.eu/research/participants/doc...,8c752a748e9138458797f2812762ac38,en,NaN,NaN,projectDeliverable,corda,project
1,101096150,https://ec.europa.eu/research/participants/doc...,72690dd7bb355fdca270488e46dff1cf,en,NaN,NaN,projectDeliverable,corda,project
2,101096150,https://ec.europa.eu/research/participants/doc...,e48d19a29dd1f9417917f29c657ed922,en,NaN,NaN,projectDeliverable,corda,project
3,101096150,https://ec.europa.eu/research/participants/doc...,63324d042781ae76e788722be75f3b8a,en,NaN,NaN,projectDeliverable,corda,project
4,101096150,https://ec.europa.eu/research/participants/doc...,8f80a9b0040e38ad9ac7c546c280cbd5,en,NaN,NaN,projectDeliverable,corda,project
...,...,...,...,...,...,...,...,...,...
23107,101052247,https://ec.europa.eu/research/participants/doc...,79f8309a0d1dc72d90fb902d3058eda1,en,NaN,NaN,projectDeliverable,corda,project
23108,101052247,https://ec.europa.eu/research/participants/doc...,9854bfeeedbc76390ac9723b5cff4ae5,en,NaN,NaN,projectDeliverable,corda,project
23109,101052247,https://ec.europa.eu/research/participants/doc...,ff48227e05101bdf2a69af99f6682513,en,NaN,NaN,projectDeliverable,corda,project
23110,101052247,https://ec.europa.eu/research/participants/doc...,e99ab3fd565565764659d9f0df929271,en,NaN,NaN,projectDeliverable,corda,project


In [58]:
# print keys for all loaded files
print_keys(SciVoc_path)
print_keys(legalBasis_path)
print_keys(organization_path)
print_keys(topics_path)
print_keys(webItems_path)
print_keys(webLink_path)


Keys for /Users/bertdepoorter/Nextcloud/EU_Horizon_Dashboard/data/raw/cordis-HORIZONprojects-csv/euroSciVoc.csv:
Index(['projectID', 'euroSciVocCode', 'euroSciVocPath', 'euroSciVocTitle',
       'euroSciVocDescription'],
      dtype='object')


Keys for /Users/bertdepoorter/Nextcloud/EU_Horizon_Dashboard/data/raw/cordis-HORIZONprojects-csv/legalBasis.csv:
Index(['projectID', 'legalBasis', 'title', 'uniqueProgrammePart'], dtype='object')


Keys for /Users/bertdepoorter/Nextcloud/EU_Horizon_Dashboard/data/raw/cordis-HORIZONprojects-csv/organization.csv:
Index(['projectID', 'projectAcronym', 'organisationID', 'vatNumber', 'name',
       'shortName', 'SME', 'activityType', 'street', 'postCode', 'city',
       'country', 'nutsCode', 'geolocation', 'organizationURL', 'contactForm',
       'contentUpdateDate', 'rcn', 'order', 'role', 'ecContribution',
       'netEcContribution', 'totalCost', 'endOfParticipation', 'active'],
      dtype='object')


Keys for /Users/bertdepoorter/Nextcloud/EU_Ho

In [42]:
read_csv_options['on_bad_lines'] = 'warn'
try:
    project_df = pd.read_csv(project_path, **read_csv_options)
    print(len(project_df.id))
except pd.errors.ParserError as e:
    print("Parsing error:", e)

15736


/var/folders/qr/g_wjhwrd58l051flw_nzj2lm0000gn/T/ipykernel_47021/2163737352.py:3: ParserWarning: Skipping line 11: Expected 20 fields in line 11, saw 22

  project_df = pd.read_csv(project_path, **read_csv_options)
/var/folders/qr/g_wjhwrd58l051flw_nzj2lm0000gn/T/ipykernel_47021/2163737352.py:3: ParserWarning: Skipping line 24: Expected 20 fields in line 24, saw 21

  project_df = pd.read_csv(project_path, **read_csv_options)
/var/folders/qr/g_wjhwrd58l051flw_nzj2lm0000gn/T/ipykernel_47021/2163737352.py:3: ParserWarning: Skipping line 80: Expected 20 fields in line 80, saw 21

  project_df = pd.read_csv(project_path, **read_csv_options)
/var/folders/qr/g_wjhwrd58l051flw_nzj2lm0000gn/T/ipykernel_47021/2163737352.py:3: ParserWarning: Skipping line 97: Expected 20 fields in line 97, saw 21

  project_df = pd.read_csv(project_path, **read_csv_options)
/var/folders/qr/g_wjhwrd58l051flw_nzj2lm0000gn/T/ipykernel_47021/2163737352.py:3: ParserWarning: Skipping line 220: Expected 20 fields in li

There are 15863 - 15737 = 126 field which could not be read. Let us inspect those lines further. 

We first log the problematic lines. There are 126 of them. Theycan be accessed by the following function. 

In [44]:
project_df, problematic_lines = inspect_bad_lines(project_path, expected_columns=20)

print(f"DataFrame loaded with {len(project_df)} rows.")
print(f"Number of problematic lines: {len(problematic_lines)}")

Found 127 problematic lines. Displaying the first 5:
Line 11: ['101114248', 'ELIA', 'CLOSED', 'Elia - Smart Assistant for English Learning', '2023-07-01', '2024-06-30', '0', '75000', 'HORIZON.3.2', 'HORIZON-EIE-2022-SCALEUP-02-02', '2023-06-05', 'HORIZON', 'HORIZON-EIE-2022-SCALEUP-02', 'HORIZON-EIE-2022-SCALEUP-02', 'HORIZON-CSA', '', "We are a startup founded exclusively by women holding the top positions CEO and CTO. Our vision is to reduce inequalities by helping people own their English. That's why we created a personal assistant – Elia. Elia is a tool for busy professionals or swamped students struggling with their English. It connects English learning to their daily activities, e.g. writing an email at work", ' watching videos on YouTube', ' or reading an article for a biology class. Because learning that is personalised and in context has been found to be the most effective form of learning. Elia started as a PhD project. Hence it\'s based on insights from cognitive linguistics

In [62]:
project_df = robust_csv_reader(project_path, expected_columns=20, problematic_column=14)

In [45]:
project_df

,id,acronym,status,title,startDate,endDate,totalCost,ecMaxContribution,legalBasis,topics,ecSignatureDate,frameworkProgramme,masterCall,subCall,fundingScheme,nature,objective,contentUpdateDate,rcn,grantDoi
0,101159220,PvSeroRDT,SIGNED,A point-of-care serological rapid diagnostic t...,2025-02-01,2030-01-31,"4062396,23","4062396,23",HORIZON.2.1,HORIZON-JU-GH-EDCTP3-2023-02-02-two-stage,2024-12-09,HORIZON,HORIZON-JU-GH-EDCTP3-2023-02-two-stage,HORIZON-JU-GH-EDCTP3-2023-02-two-stage,HORIZON-JU-RIA,,Plasmodium vivax is considered the most diffic...,2024-12-24 11:18:48,268210,10.3030/101159220
1,101096150,BIOBoost,SIGNED,Boosting innovation agencies for bioeconomy va...,2023-02-01,2025-01-31,0,500000,HORIZON.3.2,HORIZON-EIE-2022-CONNECT-01-01,2022-11-25,HORIZON,HORIZON-EIE-2022-CONNECT-01,HORIZON-EIE-2022-CONNECT-01,HORIZON-CSA,,The overall objectives of the BIOBoost project...,2022-12-01 14:09:06,243343,10.3030/101096150
2,101093997,GlycanTrigger,SIGNED,GLYCANS AS MASTER TRIGGERS OF HEALTH TO INTEST...,2023-01-01,2028-12-31,6771571,6771571,HORIZON.2.1,HORIZON-HLTH-2022-STAYHLTH-02-01,2022-12-05,HORIZON,HORIZON-HLTH-2022-STAYHLTH-02,HORIZON-HLTH-2022-STAYHLTH-02,HORIZON-RIA,,Chronic inflammation underlies several disease...,2022-12-11 19:02:29,243439,10.3030/101093997
3,101126531,CHIKVAX_CHIM,SIGNED,Late-stage clinical development of Chikungunya...,2023-06-01,2028-11-30,100000000,70000000,HORIZON.2.1,HORIZON-HLTH-2022-CEPI-15-01-IBA,2023-06-15,HORIZON,HORIZON-HLTH-2022-CEPI-15-IBA,HORIZON-HLTH-2022-CEPI-15-IBA,HORIZON-COFUND,,A Framework Partnership Agreement (FPA) betwee...,2023-09-19 19:01:01,256925,10.3030/101126531
4,101113979,The Oater,CLOSED,The Oater develops a compact machine for hyper...,2023-07-01,2023-12-31,0,75000,HORIZON.3.2,HORIZON-EIE-2022-SCALEUP-02-02,2023-06-05,HORIZON,HORIZON-EIE-2022-SCALEUP-02,HORIZON-EIE-2022-SCALEUP-02,HORIZON-CSA,,The Oater is a female-founded food tech start-...,2023-07-11 15:45:49,253030,10.3030/101113979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15858,101052410,EUCYS2022,CLOSED,EUCYS Leiden2022,2021-09-01,2023-02-28,2000000,2000000,HORIZON.4.2,HORIZON-WIDERA-2021-EUCYS-IBA,2022-02-07,HORIZON,HORIZON-WIDERA-2021-EUCYS-IBA,HORIZON-WIDERA-2021-EUCYS-IBA,HORIZON-CSA,,The main objective of this Proposal is the org...,2023-03-10 20:23:58,241771,10.3030/101052410
15859,101124648,RESAVER_2023,SIGNED,Support to Retirement Savings Vehicle for Euro...,2023-09-01,2026-08-31,"2499638,25","2499638,25",HORIZON.4.2,HORIZON-WIDERA-2023-RESAVER-IBA,2023-07-10,HORIZON,HORIZON-WIDERA-2023-RESAVER-IBA,HORIZON-WIDERA-2023-RESAVER-IBA,HORIZON-CSA,,The overall aim of the RESAVER Pension Fund as...,2023-10-13 14:43:57,257324,10.3030/101124648
15860,101052247,Leiden2022-ECS-ESOF,CLOSED,European City of Science and EuroScience Open ...,2021-08-01,2023-03-31,"3709149,25",2000000,HORIZON.4.2,HORIZON-WIDERA-2021-ESOF-IBA,2021-12-13,HORIZON,HORIZON-WIDERA-2021-ESOF-IBA,HORIZON-WIDERA-2021-ESOF-IBA,HORIZON-CSA,,The main objective of this proposal is the org...,2022-09-14 19:17:24,241770,10.3030/101052247
15861,101172981,EUCYS2024,SIGNED,European Union Contest for Young Scientists (E...,2024-02-01,2025-02-28,999500,999500,HORIZON.4.2,HORIZON-WIDERA-2024-EUCYS-IBA,2024-04-15,HORIZON,HORIZON-WIDERA-2024-EUCYS-IBA,HORIZON-WIDERA-2024-EUCYS-IBA,HORIZON-CSA,,This proposal concerns the organization of the...,2024-04-22 17:56:03,262788,10.3030/101172981


In [63]:
from etl.cleaning import clean_project

project_df = clean_project(project_df)

/Users/bertdepoorter/Nextcloud/EU_Horizon_Dashboard/backend/etl/cleaning.py:30: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(series, errors='coerce', infer_datetime_format=True)
/Users/bertdepoorter/Nextcloud/EU_Horizon_Dashboard/backend/etl/cleaning.py:30: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.to_datetime(series, errors='coerce', infer_datetime_format=True)


In [66]:
project_df['id'] = clean_numeric_column(project_df['id'])

In [63]:
from etl.cleaning import clean_string_column
project_df['status'] = clean_string_column(project_df['status'])

In [64]:
project_df['status'].unique()

array(['SIGNED', 'CLOSED', 'TERMINATED'], dtype=object)

In [66]:

project_missing = project_df.isnull()

# check which columns are missing data
for key in project_df.keys():
    missing = project_missing[project_missing[key] == True]
    if len(missing.id) > 0:
        print(f'For key {key}:\n     {len(missing.projectID)} elements are missing.')

There are apparently no missing values in this dataset. We save the dataset to the interim data folder

In [47]:
project_df.to_csv(f'{interim_dir}/project_interim.csv', sep=';')

In [59]:
project_df.keys()

Index(['id', 'acronym', 'status', 'title', 'startDate', 'endDate', 'totalCost',
       'ecMaxContribution', 'legalBasis', 'topics', 'ecSignatureDate',
       'frameworkProgramme', 'masterCall', 'subCall', 'fundingScheme',
       'nature', 'objective', 'contentUpdateDate', 'rcn', 'grantDoi'],
      dtype='object')

## Construct functions to access cleaned data

We now define some functions that allow easy access to all aspects of different projects. 


- Merge datasets into one object
- Standardize column names => they are compatible
- Create function that allow access to project-specific data:
    - function argument: project name / acronym / identifier
    - function output: data class with project information as attributes
    - Or: approach this from a class init perspective

Find some way to pass load datasets
apply class on this, without having to load the full dataset each time we initialize the class


In [109]:
# class to load datasets

class CORDIS_data():
    def __init__(self):
        '''
        Initialize class: load data from the CSV files

        set some global variables that we 

        '''
        # Load all datasets and set as class attributes
        self.data_report = pd.read_csv(f'{interim_dir}/projectdeliverables_interim.csv', delimiter=';')
        self.data_deliverables = pd.read_csv(f'{interim_dir}/projectdeliverables_interim.csv', delimiter=';')
        self.data_publications = pd.read_csv(f'{interim_dir}/projectPublications_interim.csv', delimiter=';')
        self.project_df = pd.read_csv(f'{interim_dir}/project_interim.csv', delimiter=';')
        self.sci_voc_df = pd.read_csv(SciVoc_path, delimiter=';')
        self.legal_basis_df = pd.read_csv(legalBasis_path, delimiter=';')   
        self.organization_df = pd.read_csv(organization_path, delimiter=';')
        self.topics_df = pd.read_csv(topics_path, delimiter=';')
        self.web_items_df = pd.read_csv(webItems_path, delimiter=';')
        self.web_link_df = pd.read_csv(webLink_path, delimiter=';')

        # enrich the project DataFrame with some additional information
        # Call enrichment functions
        self._enrich_temporal_features()
        self._enrich_people_and_institutions()
        self._enrich_financial_metrics()
        self._enrich_scientific_thematic()

        # Extract possible scientific fields
        self.scientific_fields = self.extract_scientific_fields()
    
    def list_of_acronyms(self, show=True):
        '''
        This function prints out a dataframe 
        '''
        acronyms = pd.DataFrame(self.project_df['acronym'].unique())
        self.acronyms = acronyms
        if show == True:
            return acronyms
    
    def _enrich_temporal_features(self):
        '''
        This function adds temporal features to the project dataframe regarding dates and durations
        '''
        print('Enriching the projects dataset with temporal information.')
        df = self.project_df.copy()
        df['startDate'] = pd.to_datetime(df['startDate'], errors='coerce')
        df['endDate'] = pd.to_datetime(df['endDate'], errors='coerce')
        df['duration_days'] = (df['endDate'] - df['startDate']).dt.days
        
        df['duration_months'] = df['duration_days'] / 30.44
        df['duration_months'] = df['duration_months'].astype(int)

        df['duration_years'] = df['duration_days'] / 365.25
        df['duration_years'] = df['duration_years'].astype(int)
        self.project_df = df
    
    def _enrich_people_and_institutions(self):
        ''' 
        This function adds some information about the people and institutions involved in the project
        Added features:
        - Number of institutions involved
        - List of institutions involved
        - Coordinator name (if role info is reliable)
        '''
        print('Enriching the projects dataset with people and institutions information.')
        # Count unique institutions per project
        orgs = self.organization_df.groupby('projectID')['name'].nunique().reset_index(name='n_institutions')

        # List all institutions
        inst_list = self.organization_df.groupby('projectID')['name'].apply(list).reset_index(name='institutions')

        # Coordinator (if role info is reliable)
        coordinators = self.organization_df[self.organization_df['role'].str.lower() == 'coordinator']
        pi_names = coordinators.groupby('projectID')['name'].first().reset_index(name='coordinator_name')

        # Merge all
        self.project_df = self.project_df.merge(orgs, how='left', left_on='id', right_on='projectID')
        self.project_df = self.project_df.merge(inst_list, how='left', left_on='id', right_on='projectID')
        self.project_df = self.project_df.merge(pi_names, how='left', left_on='id', right_on='projectID')

    def _enrich_financial_metrics(self):
        '''
        This function adds some financial metrics to the final project dataframe
        Added feautures:
        - EC contribution per year
        - Total cost per year

        '''
        print('Enriching the projects dataset with financial information.')
        df = self.project_df.copy()

        # Convert to numeric
        df['ecMaxContribution'] = pd.to_numeric(df['ecMaxContribution'], errors='coerce')
        df['totalCost'] = pd.to_numeric(df['totalCost'], errors='coerce')

        # Annualized budget (if duration is available)
        if 'duration_years' not in df.columns:
            self._enrich_temporal_features()
            df = self.project_df

        df['ecContribution_per_year'] = df['ecMaxContribution'] / df['duration_years']
        df['totalCost_per_year'] = df['totalCost'] / df['duration_years']

        self.project_df = df
    
    def _enrich_scientific_thematic(self):
        ''' 
        Adds scientific and thematic information to the project dataframe:
        - List of full euroSciVocPath strings per project
        - List of euroSciVocTitle values per project
        - List of topic titles from the topics.csv file
        '''
        print('Enriching the projects dataset with thematic / scientific information.')
        # Full paths (euroSciVocPath) per project
        sci_paths = self.sci_voc_df.groupby('projectID')['euroSciVocPath'].apply(list).reset_index(name='sci_voc_paths')

        # Titles per project
        sci_titles = self.sci_voc_df.groupby('projectID')['euroSciVocTitle'].apply(list).reset_index(name='sci_voc_titles')

        # Topics from topics.csv
        topic_titles = self.topics_df.groupby('projectID')['title'].apply(list).reset_index(name='topic_titles')

        # Merge into project_df 
        self.project_df = self.project_df.drop(columns=['projectID']).merge(sci_titles, how='left', left_on='id', right_on='projectID')
        self.project_df = self.project_df.drop(columns=['projectID']).merge(sci_paths, how='left', left_on='id', right_on='projectID')
        self.project_df = self.project_df.drop(columns=['projectID']).merge(topic_titles, how='left', left_on='id', right_on='projectID')


    def get_projects_by_scientific_field(self, keyword):
        """
        Get a list of all projects filtered by the scientific field. 
        """
        # initialize empty dictionary with list in which we store project acronyms
        projects_per_field = {}
        for field in self.scientific_fields:
            projects_per_field[str(field)] = []

        # Go through  sciVoc dataframe and add acronym to the list
        for i in range(len(self.sci_voc_df)):
            if keyword in self.sci_voc_df['euroSciVocPath'][i]:
                project_id = self.sci_voc_df['projectID'][i]
                acronym = self.project_df[self.project_df['id'] == project_id]['acronym'].values[0]
                projects_per_field[str(keyword)].append(acronym)
                
        # Remove duplicates
        projects_per_field[str(keyword)] = list(set(projects_per_field[str(keyword)]))
        return projects_per_field

    def get_projects_by_institution(self, institution_keyword):
        filtered = self.organization_df[self.organization_df['name'].str.contains(institution_keyword, case=False, na=False)]
        acronyms = filtered['projectAcronym'].dropna().unique().tolist()
        return acronyms
    
    def extract_scientific_fields(self):
        paths = self.sci_voc_df['euroSciVocPath'].dropna().unique()
        fields = set()
        for path in paths:
            segments = path.strip('/').split('/')
            if segments:
                fields.add(segments[0])
        return sorted(fields)

    def export_dataframes(self, directory, format='csv', include_all=False):
        """
        Export enriched project_df and optionally all loaded dataframes.

        Parameters:
        - directory: str. Path where files will be saved.
        - format: 'csv' or 'excel' (default: 'csv')
        - include_all: if True, export all loaded dataframes; else only project_df
        """
        def _save(df, name):
            path = os.path.join(directory, f"{name}.{ext}")
            if format == 'csv':
                df.to_csv(path, index=False)

        if format == 'csv':
            ext = 'csv'
        else:
            print('Use CSV dumbass. Proceeding to store the data as CSV files.')
            ext = 'csv'
        _save(self.project_df, "project_df")

        if include_all:
            _save(self.data_deliverables, "data_deliverables")
            _save(self.data_publications, "data_publications")
            _save(self.organization_df, "organization_df")
            _save(self.legal_basis_df, "legal_basis_df")
            _save(self.topics_df, "topics_df")
            _save(self.sci_voc_df, "sci_voc_df")
            _save(self.web_items_df, "web_items_df")
            _save(self.web_link_df, "web_link_df")


In [110]:
Data_structure = CORDIS_data()

Enriching the projects dataset with temporal information.
Enriching the projects dataset with people and institutions information.
Enriching the projects dataset with financial information.
Enriching the projects dataset with thematic / scientific information.


In [ ]:
# store feature-enriched dataframe to the processed directory
Data_structure.export_dataframes(f'{processed_dir}/')

In [72]:
Data_structure.list_of_acronyms()

,0
0,PvSeroRDT
1,BIOBoost
2,GlycanTrigger
3,CHIKVAX_CHIM
4,The Oater
...,...
15048,EUCYS2022
15049,RESAVER_2023
15050,Leiden2022-ECS-ESOF
15051,EUCYS2024


In [95]:
class Project_data(CORDIS_data):
    def __init__(self, project_id=None, acronym=None):
        # Inherit from CORDIS_data by initialzing parent class
        super().__init__()

        # Check if both project_id and acronym are provided
        self.id, self.acronym = self._resolve_project_id_acronym(project_id, acronym)

        # Add all project-specific data as attributes
        
        self.project_info = self._get_project_info()
        self.publications = self._get_publications()
        self.deliverables = self._get_deliverables()
        self.organizations = self._get_organizations()
        self.scivoc = self._get_scivoc()
        self.topics = self._get_topics()
        self.legal_basis = self._get_legal_basis()

        # Add some enriched data
        self.temporal_features = self._compute_temporal_features()
        self.people_institutions = self._compute_people_institutions()
        self.financial_metrics = self._compute_financial_metrics()
        self.scientific_thematic = self._compute_scientific_thematic()

    def _resolve_project_id_acronym(self, project_id, acronym):
        if project_id is not None and acronym is not None:
            expected_acronym = self.project_df[self.project_df['id'] == project_id]['acronym'].values[0]
            if expected_acronym != acronym:
                raise ValueError(f"Acronym mismatch: ID {project_id} is linked to {expected_acronym}, not {acronym}.")
        elif acronym is not None:
            project_id = self.project_df[self.project_df['acronym'] == acronym]['id'].values[0]
        elif project_id is not None:
            acronym = self.project_df[self.project_df['id'] == project_id]['acronym'].values[0]
        else:
            raise ValueError("Provide at least one of project_id or acronym.")
        return project_id, acronym

    def _get_project_info(self):
        return self.project_df[self.project_df['id'] == self.id].iloc[0].to_dict()

    def _get_publications(self):
        return self.data_publications[self.data_publications['projectID'] == self.id]

    def _get_deliverables(self):
        return self.data_deliverables[self.data_deliverables['projectID'] == self.id]

    def _get_organizations(self):
        return self.organization_df[self.organization_df['projectID'] == self.id]

    def _get_scivoc(self):
        return self.sci_voc_df[self.sci_voc_df['projectID'] == self.id]

    def _get_topics(self):
        return self.topics_df[self.topics_df['projectID'] == self.id]

    def _get_legal_basis(self):
        return self.legal_basis_df[self.legal_basis_df['projectID'] == self.id]
    

    # Add additional project features
    def _compute_temporal_features(self):
        fmt = "%Y-%m-%d"
        start = self.project_info.get("startDate", None)
        end = self.project_info.get("endDate", None)
        ec_sig = self.project_info.get("ecSignatureDate", None)

        try:
            start_date = datetime.strptime(start, fmt)
            end_date = datetime.strptime(end, fmt)
            duration_days = (end_date - start_date).days
        except:
            duration_days = None

        return {
            "start_year": start.year if start else None,
            "end_year": end.year if end else None,
            "signature_year": ec_sig[:4] if ec_sig else None,
            "duration_days": duration_days
        }
    
    def _compute_people_institutions(self):
        orgs = self.organizations
        if orgs.empty:
            return {}
        country_counts = orgs["country"].value_counts().to_dict()
        activity_types = orgs["activityType"].value_counts().to_dict()
        n_partners = orgs["organisationID"].nunique()

        return {
            "n_partners": n_partners,
            "countries": country_counts,
            "activity_types": activity_types
        }

    def _compute_financial_metrics(self):
        ec_total = self.project_info.get("ecMaxContribution", None)
        total_cost = self.project_info.get("totalCost", None)
        ec_partner_sum = self.organizations["ecContribution"].sum()
        cost_partner_sum = self.organizations["totalCost"].sum()

        try:
            ec_per_deliverable = ec_total / len(self.deliverables)
        except:
            ec_per_deliverable = None

        try:
            ec_per_publication = ec_total / len(self.publications)
        except:
            ec_per_publication = None

        return {
            "ec_total": ec_total,
            "total_cost": total_cost,
            "ec_sum_from_partners": ec_partner_sum,
            "cost_sum_from_partners": cost_partner_sum,
            "ec_per_deliverable": ec_per_deliverable,
            "ec_per_publication": ec_per_publication
        }

    def _compute_scientific_thematic(self):
        scivoc_titles = self.scivoc['euroSciVocTitle'].dropna().unique().tolist()
        topic_titles = self.topics['title'].dropna().unique().tolist()

        pub_types = self.publications['isPublishedAs'].value_counts().to_dict()
        deliverable_types = self.deliverables['deliverableType'].value_counts().to_dict()

        return {
            "scivoc_keywords": scivoc_titles,
            "topic_keywords": topic_titles,
            "publication_types": pub_types,
            "deliverable_types": deliverable_types
        }
    def summary(self):
        return {
            "project_id": self.id,
            "acronym": self.acronym,
            "title": self.project_info.get("title", ""),
            "temporal": self.temporal_features,
            "institutions": self.people_institutions,
            "financials": self.financial_metrics,
            "keywords": self.scientific_thematic
        }
    
    def inspect_project_data(self):
        """
        Print or return a structured overview of all enriched data for the selected project.
        """
        if not hasattr(self, 'id') or not hasattr(self, 'acronym'):
            raise AttributeError("Please set a project using the `project()` method first.")

        print(f"\nProject: {self.acronym} (ID: {self.id})")
        print("="*60)

        print("\nPublications:")
        pprint(getattr(self, 'publications', {}), indent=4)

        print("\nDeliverables:")
        pprint(getattr(self, 'deliverables', {})[['deliverableType', 'description']], indent=4)

        print("\nInstitutions / Organizations:")
        pprint(getattr(self, 'organizations', []), indent=4)

        print("\nFinancial Info:")
        pprint({
            'Total Cost': getattr(self, 'total_cost', None),
            'EC Contribution': getattr(self, 'ec_contribution', None),
            'Num Orgs': getattr(self, 'num_organizations', None),
            'Countries': getattr(self, 'countries', None),
        }, indent=4)

        print("\nDates & Duration:")
        pprint({
            'Start Date': getattr(self, 'start_date', None),
            'End Date': getattr(self, 'end_date', None),
            'Duration (days)': getattr(self, 'duration_days', None),
            'Year': getattr(self, 'year', None)
        }, indent=4)

        print("\nLegal & Administrative:")
        pprint({
            'Legal Basis': getattr(self, 'legal_basis', None),
            'Funding Scheme': getattr(self, 'funding_scheme', None),
            'Framework Programme': getattr(self, 'framework_programme', None),
        }, indent=4)

        print("\n🔬 Scientific Keywords (euroSciVoc):")
        pprint(getattr(self, 'sci_keywords', []), indent=4)

        print("\nProject Topics:")
        pprint(getattr(self, 'topics', []), indent=4)

        print("\nWeb Links:")
        pprint(getattr(self, 'web_links', []), indent=4)


In [90]:
p = Project_data(acronym="CLIMB")
summary = p.summary()
print(summary["financials"])

{'ec_total': 1622273.0, 'total_cost': 1622273.0, 'ec_sum_from_partners': np.float64(1622273.0), 'cost_sum_from_partners': '1622273', 'ec_per_deliverable': None, 'ec_per_publication': None}


Use pprint to get out the background information in a readable format.

In [91]:
pprint(summary["financials"])

{'cost_sum_from_partners': '1622273',
 'ec_per_deliverable': None,
 'ec_per_publication': None,
 'ec_sum_from_partners': np.float64(1622273.0),
 'ec_total': 1622273.0,
 'total_cost': 1622273.0}


In [96]:
# Inspect a certain project
p = Project_data(acronym="BIOBoost")
p.inspect_project_data()


Project: BIOBoost (ID: 101096150)

Publications:
Empty DataFrame
Columns: [Unnamed: 0, id, title, isPublishedAs, authors, journalTitle, journalNumber, publishedYear, publishedPages, issn, isbn, doi, projectID, projectAcronym, collection, contentUpdateDate, rcn]
Index: []

Deliverables:
                              deliverableType  \
18191                      Documents, reports   
18192                      Documents, reports   
18193  Websites, patent fillings, videos etc.   
18194                      Documents, reports   
18195                    Data Management Plan   
18196                      Documents, reports   

                                             description  
18191  The project management handbook will provide c...  
18192  The PDEC provides information to all project p...  
18193  Interactive online tool showing and mapping ma...  
18194  The report includes information on innovation ...  
18195  The DMP provides clear information on the cons...  
18196  Assessm